In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer, gen_rbf_X, create_clus_split
from spe.relaxed_lasso import RelaxedLasso

In [2]:
err_cmp = ErrorComparer()

In [78]:
niter = 300

n=50**2
p=30
s=30

k=10

In [79]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()

In [80]:
rho = 0.5
delta = 0.8

D = np.abs(c_x - c_x[:,None]) + np.abs(c_y - c_y[:,None])
Sigma_t = rho**D
Sigma_t = delta*Sigma_t + (1-delta) * np.eye(n)

# m = 20
# c = np.zeros(n)
# c[:m] = [rho**i for i in np.arange(m)]
# Sigma_t = toeplitz(c)

# b = n / m
# c = [rho**i for i in np.arange(m)] 
# Sigma_t = block_diag(*[toeplitz(c) for _ in np.arange(b)])

Chol_t = np.linalg.cholesky(Sigma_t)
# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

In [81]:
# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

X = gen_rbf_X(c_x, c_y, p)
# X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [82]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [96]:
(test_err,
 kfcv_err,
 spcv_err,
 lin_err) = err_cmp.compareLinearTrTs(n=n,
                                     p=p,
                                     X=X,
                                     beta=beta,
                                     coord=np.stack([c_x, c_y]).T,
                                     Chol_t=Chol_t,
                                     niter=niter,
                                     k=k,
                                     tr_frac=.07)
    

0
0.0696
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [97]:
# fig_X = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=X[:,13], opacity=0.50)])
# fig_X.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Feature Xj'),
# )
# fig_X.show()

In [98]:
# fig_y = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=y, opacity=0.50)])
# fig_y.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Response Y'),
# )
# fig_y.show()

In [99]:
# fig_mu = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=mu, opacity=0.50)])
# fig_mu.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Mean'),
# )
# fig_mu.show()

In [100]:
# fig_eps = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=y-mu, opacity=0.50)])
# fig_eps.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Eps'),
# )
# fig_eps.show()

In [101]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [102]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [103]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
risk_lin = lin_err.mean()
risk, risk_kfcv, risk_spcv, risk_lin

(7316.282573604084,
 0.0010518916143365897,
 0.02292893773293611,
 5564.8280748288535)

In [104]:
df = pd.DataFrame({'GenCp': (lin_err.T),# - test_err.T) / test_err.T,
                   'KFCV': (kfcv_err.T),# - test_err.T) / test_err.T,
                   'SPCV': (spcv_err.T)})# - test_err.T) / test_err.T})

In [105]:
fig_lin = px.box((df - risk) / risk, 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="lin correction",
             points=False)
fig_lin.update_traces(boxmean=True)

fig_lin.add_hline(y=0., line_color='red')

In [106]:
fig = go.Figure(data=go.Scatter(
        x=[0, 1, 2],
        y=((df - risk)/risk).mean(),
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=((df - risk)/risk).std(),
            visible=True)
    ))
fig.show()

In [107]:
(df/risk).mean()

GenCp    7.606087e-01
KFCV     1.437741e-07
SPCV     3.133960e-06
dtype: float64

In [108]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="Linear Regression Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
fig.show()